In [1]:
import pandas as pd
import numpy as np
from collections import Counter 

#Visualization
import matplotlib.pyplot as plt 
import os
#Model and performance
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.svm import OneClassSVM
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from keras.models import Model, Sequential
from keras.layers import Dense, Input


In [ ]:
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense

# Load the dataset
data = pd.read_csv('system-1.csv')
train_size = int(len(data) * 0.5)

# Use the first 50% of the data for training

df = data.iloc[:, 1:]
X_train = df.iloc[:train_size]

# Autoencoder dimensions
input_dim = X_train.shape[1]  # Number of features
encoding_dim = 10        # Latent space dimensionality

# Define the Autoencoder using the Functional API
input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Full Autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoded)

# Compile and train the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, verbose=0)

# Extract the Encoder model for feature extraction
encoder = Model(inputs=input_layer, outputs=encoded)

# Use the encoder to transform the data into the latent space
encoded_data = encoder.predict(X_train)
print("Encoded data shape:", encoded_data.shape)


1340/1340 ━━━━━━━━━━━━━━━━━━━━ 0s 195us/step
Encoded data shape: (42874, 10)


In [3]:
svm_model = OneClassSVM(kernel='rbf', gamma='scale', nu=0.1)
svm_model.fit(encoded_data)

OneClassSVM(nu=0.1)

In [4]:
test_input_dim = df.shape[1]  
test_encoding_dim = 10        

test_input_layer = Input(shape=(input_dim,))
test_encoded = Dense(test_encoding_dim, activation='relu')(test_input_layer)
test_decoded = Dense(test_input_dim, activation='sigmoid')(test_encoded)

test_autoencoder = Model(inputs=test_input_layer, outputs=test_decoded)

test_autoencoder.compile(optimizer='adam', loss='mse')
test_autoencoder.fit(df, df, epochs=50, batch_size=32, verbose=0)

test_encoder = Model(inputs=test_input_layer, outputs=test_encoded)

# Use the encoder to transform the data into the latent space
test_encoded_data = test_encoder.predict(df)
print("Encoded data shape:", test_encoded_data.shape)

2680/2680 ━━━━━━━━━━━━━━━━━━━━ 1s 209us/step
Encoded data shape: (85749, 10)


In [5]:
#svm_model.fit(test)
pred = svm_model.predict(test_encoded_data)

In [8]:
decision_function = svm_model.decision_function(test_encoded_data)
threshold = np.percentile(decision_function, 5)
adjusted_predictions = (decision_function < threshold).astype(int)

boolean_anomalies = adjusted_predictions == 0
adjusted_anomalies = np.sum(boolean_anomalies == False)
adjusted_normal_points = np.sum(adjusted_predictions == True)

print(f"Number of anomalies with adjusted threshold: {adjusted_anomalies}")
print(f"Number of normal points with adjusted threshold: {adjusted_normal_points}")

# Return the boolean anomalies array
print("Boolean anomalies array:", boolean_anomalies)

Number of anomalies with adjusted threshold: 4288
Number of normal points with adjusted threshold: 4288
Boolean anomalies array: [ True  True  True ...  True  True  True]


Number of anomalies with adjusted threshold: 4288
Number of normal points with adjusted threshold: 81461
